# National Health Insurance (NHI) – Predicting Healthcare Service Demand (South Africa)

**Technical Programming 2 – Assessment 4**  
**Submission Date:** 26 August 2025  

---

## Group & Admin Cell (Alphabetical by Surname)

- Ncobeni, N.A – 22412310  
- Nikwe, T – 22309434  
- Ngeleka, S.K – 22221194  
- Mathonsi, S.S – 22339617  
- Mazibuko, S.R – 22000786  
- Sigwaza, S.N – 22460979  
- Zungula, E – 22339408  

---

## Task Choice
We selected **TASK ONE: Predicting Healthcare Service Demand**.  

### Why this task?
- Aligns directly with NHI rollout needs: anticipating demand to allocate funding and capacity.  
- South African public microdata with health modules and demographics are publicly available (e.g., **GHS 2024**).  
- Natural, supervised-learning framing with clear targets (e.g., probability of clinic/hospital visit).  


## Dataset Sources (Credibility & Validity)

For this project we used **open, credible, multi-relational datasets** from official sources:

1. **General Household Survey 2024 (Stats SA)**  
   - Repository: World Bank Microdata Library (Released 8 July 2025)  
   - Link: [https://microdata.worldbank.org/index.php/catalog/6792](https://microdata.worldbank.org/index.php/catalog/6792)

2. **General Household Survey Overview (Stats SA)**  
   - Link: [https://www.statssa.gov.za/?page_id=16405](https://www.statssa.gov.za/?page_id=16405)

3. **South Africa Health Facility Master List (Nature Scientific Data)**  
   - Maina et al., 2019. “A spatial database of health facilities …”  
   - Link: [https://www.nature.com/articles/s41597-019-0142-2](https://www.nature.com/articles/s41597-019-0142-2)


---

## Project Analysis (Start Here)
Use the following sections for your dataset loading, cleaning, visualization, machine learning, and evaluation.  
All code cells and outputs should follow after this point.
